In [18]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
import aot

In [19]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)  

settings_root_path = base_dir / core_settings["paths"]["settings_path"] 
#video_db_path = base_dir / 'data/videos/database_originals.tsv'                 diffenrent path for original videos
#video_db = pd.read_csv(video_db_path, sep='\t')                 ########################################################

In [20]:

'''
videos_names = []
for i in range(1,len(video_db)):
    if video_db['grade'][i] != 'k' and video_db['grade'][i] != 'l':   #video_db['grade'][i] == 'j' or video_db['grade'][i] == 'NA':
        videos_names.append(video_db['video_name'][i])
'''
#                                                           select the original videos name that we will finally use                                                            
print("videos number: ", len(videos_names))

origin_videos_number = len(videos_names)

subject_number = core_settings['various']['subject_number']
session_number = core_settings['various']['session_number']
run_number = core_settings['various']['run_number']
blank_number_per_run = core_settings['various']['blank_number_per_run']#only for main run
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
video_number_per_run_main = int((unique_video_number_per_session*4) / run_number)

print("videos names: ", videos_names)
print("subject number: ", subject_number)
print("session number: ", session_number)
print("run number: ", run_number)
print("unique video number per session: ", unique_video_number_per_session)



videos number:  1548
videos names:  ['yt-Q4edV5qiqu0_578.mp4', 'bing-www_break_com_video_like-a-cat-catching-a-fly-3131008_12.mp4', 'yt-IX6uxR5DSKk_2.mp4', 'yt-Fdv-V--QlR0_117.mp4', 'yt-LUIvP_NDWAs_45.mp4', 'yt-l_ioasf6yHg_44.mp4', 'yt-xKteiAi5h7Q_36.mp4', 'yt-E8EDQCPbBeI_196.mp4', 'yt-GbL63AmOIss_116.mp4', 'yt-7W50lcw5FCo_37.mp4', 'yt-5rC42egLl5E_387.mp4', 'Xq0a2BWE3zA_53.mp4', 'yt-kRR0CluqaX4_130.mp4', 'yt-8LO59eN9om4_16.mp4', 'yt-McqS-QObQbk_122.mp4', 'yt-e_eOzV7qMZo_374.mp4', 'yt-xlHdGsXVZFU_83.mp4', 'yt-tEzCpMrjOHs_24.mp4', 'yt-Xq6uX1jfFRk_21.mp4', 'yt-eF5J5Kjt6oY_23.mp4', 'yt-vQ-rd-1gwH4_238.mp4', 'production_ID_4707837.mp4', 'production_ID_4510883.mp4', 'pexels-pavel-danilyuk-8626652.mp4', 'pexels-roman-odintsov-8231493.mp4', 'pexels-mart-production-8435081.mp4', 'pexels-cottonbro-6789228.mp4', 'production_ID_4540332.mp4', 'pexels-anastasia-shuraeva-6014070.mp4', 'production_ID_4154238.mp4', 'pexels-kindel-media-8165572.mp4', 'production_ID_4122384.mp4', 'pexels-tima-miroshniche

In [21]:
def make_settings_for_one_session(
    subject_number, session_number, original_video_names, exp_tupe="main"
):  # input videos that before selection
    """
    original_video_names here should also be the uniqe original video names that will only be used in one session.
    the selection work should be done before this function (in the make_settings_for_one_subject function)
    """

    def make_settings_for_one_run(
        subject_number, session_number, run_number, video_names, exp_type="main"
    ):  # input videos that after selection
        """
        video_names: the video names that will be used directly used in the experiment,
        already selected, resampled, reversed, and doubled for a run
        this fuction just add some blanks into the video_names and save settings for one run
        """
        # i is run number
        settings = copy.deepcopy(stimuli_settings_temp)
        settings["stimuli"]["movie_files"] = video_names
        # uniformly distribute the blanks into movies and add some perturbations
        movie_number_per_blank = int(
            len(settings["stimuli"]["movie_files"]) / blank_number_per_run
        )
        for i in range(blank_number_per_run):
            perturb = random.randint(-1, 1)
            settings["stimuli"]["movie_files"].insert(
                (i + 1) * movie_number_per_blank + i + perturb, "blank"
            )

        # save settings
        with open(
            str(settings_root_path)
            + "/"
            + exp_type
            + "/experiment_settings_"
            + "sub_"
            + subject_number.zfill(2)
            + "_ses_"
            + session_number.zfill(2)
            + "_run_"
            + run_number.zfill(2)
            + ".yml",
            "w",
        ) as outfile:
            print(
                str(settings_root_path)
                + "/"
                + exp_type
                + "/experiment_settings_"
                + "sub_"
                + subject_number.zfill(2)
                + "_ses_"
                + session_number.zfill(2)
                + "_run_"
                + run_number.zfill(2)
                + ".yml",
                "w",
            )
            print(settings)
            print(" ")
            yaml.dump(settings, outfile, default_flow_style=False)
        return settings

    original_video_names = copy.deepcopy(original_video_names)

    # now the original_video_names are the names that will only be used in one session, don't need to be sub-sampled for each session

    resampled_video_names = [
        name.replace(".mp4","_fw.mp4") for name in original_video_names
    ]  # for gradin and labeling
    reversed_video_names = [
        name.replace(".mp4","_rev.mp4") for name in resampled_video_names
    ]  ##################################################################
    total_video_names = copy.deepcopy(resampled_video_names) + copy.deepcopy(
        reversed_video_names
    )
    doubled_total_video_names = copy.deepcopy(total_video_names) + copy.deepcopy(
        total_video_names
    )  # for main exp
    random.shuffle(total_video_names)
    random.shuffle(doubled_total_video_names)

    for i in range(1, run_number + 1):
        if i == run_number:
            # make settings for main run
            video_names = doubled_total_video_names[
                (i - 1) * video_number_per_run_main :
            ]
            make_settings_for_one_run(
                subject_number, session_number, str(i), video_names, exp_type="main"
            )
        else:
            # make settings for main run
            video_names = doubled_total_video_names[
                (i - 1) * video_number_per_run_main : i * video_number_per_run_main
            ]
            make_settings_for_one_run(
                subject_number, session_number, str(i), video_names, exp_type="main"
            )

In [22]:
def make_settings_for_one_subject(subject_number,original_video_names,exp_tupe = "main"):
    original_video_names = copy.deepcopy(original_video_names)
    #first shuffle the original video names for each subject
    random.shuffle(original_video_names)
    for i in range(1,session_number+1):
        #get i th {unique_video_number_per_session} videos for each session
        selected_video_names = original_video_names[(i-1)*unique_video_number_per_session:i*unique_video_number_per_session]
        make_settings_for_one_session(subject_number,str(i),selected_video_names,exp_tupe = exp_tupe)

def make_settings_for_all_subjects(original_video_names,exp_tupe = "main"): 
    for i in range(1,subject_number+1):
        make_settings_for_one_subject(str(i),original_video_names,exp_tupe = exp_tupe)

In [23]:
make_settings_for_all_subjects(videos_names,exp_tupe = "main")

/Users/shufanzhang/Documents/PhD/Arrow_of_time/arrow_of_time/aot/data/experiment/settings/main/experiment_settings_sub_01_ses_01_run_01.yml w
{'stimuli': {'movie_files': ['R_S_yt-AbJBxqEYoMg_27.mp4', 'R_S_yt-GjFQxAklviE_25.mp4', 'S_pexels-kindel-media-8165572.mp4', 'S_IrGYVLkWeQs_9.mp4', 'R_S_yt-uSHBdUi8_V0_24.mp4', 'S_yt-dO0OuozAgt0_225.mp4', 'blank', 'R_S_pexels-laker-5793000.mp4', 'R_S_pexels-tima-miroshnichenko-8724308.mp4', 'S_yt-K_gHa2x2OQA_319.mp4', 'S_pexels-mikhail-nilov-8410084.mp4', 'S_yt-Jo6PxivsTOg_247.mp4', 'S_yt-F3hhts3bwWg_25.mp4', 'R_S_yt-6aqg1P9YqR8_38.mp4', 'blank', 'R_S_yt-BAQ3Xhm_PPE_23.mp4', 'R_S_yt-sPhtPyQM00c_106.mp4', 'S_yt-mL3MuCfKtdY_79.mp4', 'S_yt-qurTr1BJMec_244.mp4', 'blank', 'R_S_ILSVRC2015_train_00165009.mp4', 'S_yt-kINZMjys3O4_112.mp4', 'R_S_yt-0xzYSBvVfYM_197.mp4', 'S_yt-EAMpitsjbbA_153.mp4', 'S_yt-eF5J5Kjt6oY_23.mp4', 'R_S_ILSVRC2015_train_00559000.mp4', 'R_S_Pexels_Videos_2791797.mp4', 'blank', 'S_ILSVRC2015_train_00369000.mp4', 'S_yt-2lnxIdOTxog_157

In [24]:
'''
f, s = plt.subplots(1, 1, figsize=(16,4))
rand_YTBBs, rand_MITs, randPXLs = np.random.randint(16,136, size=20), np.random.randint(16,136, size=80), np.random.randint(16,136, size=15)
for i, rnds in enumerate([rand_YTBBs, rand_MITs, randPXLs]):
    for r in rnds:
        s.axvline(r, c=['r', 'g', 'b'][i], lw=3)
s.set_ylim([-0.1,1.1])
'''


"\nf, s = plt.subplots(1, 1, figsize=(16,4))\nrand_YTBBs, rand_MITs, randPXLs = np.random.randint(16,136, size=20), np.random.randint(16,136, size=80), np.random.randint(16,136, size=15)\nfor i, rnds in enumerate([rand_YTBBs, rand_MITs, randPXLs]):\n    for r in rnds:\n        s.axvline(r, c=['r', 'g', 'b'][i], lw=3)\ns.set_ylim([-0.1,1.1])\n"